In [1]:
from tsai.models import InceptionTime, TST, ResNet
# from tsai.data.core import get_UCR_data
# from tsai.learner import TSClassifier
import os
import numpy as np
import sys
print(sys.executable)
import sys
sys.path.insert(1, '../src/')
from CNN_classifier_model import CNNClassifier, train_model, evaluate_model
from config import raw_data_path, univariate_data_path, processed_data_path
from preprocessing_modules import create_time_windows_with_labels
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import wandb
wandb.login(key="5f15eb7efc1e0e939ccc83345338a0b8c24e2fbc")
from fastai.optimizer import SGD


/home/nwertheim/miniconda3/bin/python


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nwertheim/.netrc
wandb: Currently logged in as: noorwertheim to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
'''MY DATA'''
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train_2.npy")
train_data = np.load(train_dir, allow_pickle=True)
# train_data = pd.DataFrame(train_data)
test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test_2.npy")
test_data = np.load(test_dir, allow_pickle=True)
# test_data = pd.DataFrame(test_data)
print(len(train_data))
print(len(test_data))
print(train_data[0])
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.utils.class_weight import compute_class_weight

from collections import Counter

# Example dataset with windows and labels
train_windows, train_labels = create_time_windows_with_labels(train_data)
test_windows, test_labels = create_time_windows_with_labels(test_data)

# Count label distribution
train_label_counts = Counter(train_labels)
test_label_counts = Counter(test_labels)

# Print the counts
print("Train label distribution:")
print(f"  Term (0): {train_label_counts[0]}")
print(f"  Preterm (1): {train_label_counts[1]}")

print("\nTest label distribution:")
print(f"  Term (0): {test_label_counts[0]}")
print(f"  Preterm (1): {test_label_counts[1]}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

train_labels_tensor = train_labels_tensor.long()
test_labels_tensor = test_labels_tensor.long()

744
192
{'record_name': 'Hopper-2024_11_03_14_13_16-0000010179-0002-chan0', 'signal': array([[ 0.70569938],
       [ 1.58579907],
       [ 1.22047813],
       ...,
       [-0.30476434],
       [-0.16021862],
       [-0.40381152]]), 'fs': 20, 'preterm': 1}
Train label distribution:
  Term (0): 3342
  Preterm (1): 6558

Test label distribution:
  Term (0): 408
  Preterm (1): 1512


In [3]:
import torch
import optuna
import numpy as np
from tsai.models.ResNet import ResNet
from fastai.learner import Learner
from fastai.data.core import DataLoaders
from fastai.losses import CrossEntropyLossFlat
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, Subset
from sklearn.metrics import average_precision_score
from fastai.metrics import Metric
from fastai.callback.wandb import WandbCallback

# Create TensorDataset from your data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

# ===== Compute class weights =====
train_labels_np = train_labels_tensor.numpy()
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_labels_np), y=train_labels_np)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

print(len(train_windows_tensor))
# Custom Average Precision Metric
class AveragePrecision(Metric):
    def __init__(self):
        self.pred = []
        self.target = []

    def reset(self):
        self.pred, self.target = [], []

    def accumulate(self, learn):
        preds = learn.pred.detach().softmax(dim=-1)[:, 1]  # Get probabilities for class 1
        targs = learn.y.detach()
        self.pred.append(preds.cpu())
        self.target.append(targs.cpu())

    @property
    def value(self):
        preds = torch.cat(self.pred).numpy()
        targs = torch.cat(self.target).numpy()
        return average_precision_score(targs, preds)

    @property
    def name(self): return "avg_precision"

def objective(trial):
    print('Trial:', trial)
    print(f"Starting trial {trial.number}")

    learning_rate = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    avg_precisions = []

    for train_idx, val_idx in kf.split(np.arange(len(dataset))):
        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)

        dls = DataLoaders.from_dsets(
            train_dataset,
            val_dataset,
            bs=batch_size,
            shuffle=True,
            num_workers=0
        )

        model = ResNet(c_in=1, c_out=2).to(device)

        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

        learn = Learner(
            dls, model,
            loss_func=loss_func,
            opt_func=SGD,
            metrics=AveragePrecision()
        )

        learn.fit_one_cycle(5, lr_max=learning_rate)
        ap = learn.validate()[1]  
        avg_precisions.append(ap)
        

    return np.mean(avg_precisions)

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

print("Best trial:")
print(study.best_trial)


# Save the study
import joblib
joblib.dump(study, "resnet_baseline_optuna_SGD.pkl")

/home/nwertheim/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-08 14:25:26,815] A new study created in memory with name: no-name-932aa9ff-dc74-481c-9249-f4aec4adc347


9900
Trial: <optuna.trial._trial.Trial object at 0x14538064bda0>
Starting trial 0


epoch,train_loss,valid_loss,avg_precision,time
0,0.719862,0.701664,0.735374,00:09
1,0.712505,0.705989,0.760265,00:08
2,0.702987,0.695779,0.746674,00:08
3,0.704614,0.698705,0.760288,00:08
4,0.696119,0.700163,0.741564,00:08


epoch,train_loss,valid_loss,avg_precision,time
0,0.714238,0.709556,0.644092,00:08
1,0.708251,0.703233,0.683578,00:08
2,0.708412,0.701472,0.623502,00:08
3,0.706472,0.703610,0.693846,00:08
4,0.700852,0.698142,0.647477,00:08


epoch,train_loss,valid_loss,avg_precision,time
0,0.692768,0.695318,0.696071,00:08
1,0.691521,0.694248,0.693097,00:08
2,0.692840,0.693901,0.694700,00:08
3,0.693933,0.694145,0.695093,00:08
4,0.695062,0.694184,0.699990,00:08


...

epoch,train_loss,valid_loss,avg_precision,time
0,0.692326,0.690707,0.751234,00:08
1,0.686605,0.686188,0.746490,00:08
2,0.687179,0.684875,0.747131,00:08
3,0.689264,0.684611,0.746860,00:08
4,0.685026,0.684266,0.749039,00:08


epoch,train_loss,valid_loss,avg_precision,time
0,0.690928,0.690764,0.745284,00:08
1,0.690038,0.687493,0.722932,00:08
2,0.689008,0.686534,0.724207,00:08
3,0.688104,0.689212,0.760833,00:08
4,0.683761,0.689474,0.747174,00:08


[I 2025-05-08 15:55:19,987] Trial 24 finished with value: 0.7433263067708653 and parameters: {'lr': 0.0003808581640053048, 'batch_size': 16}. Best is trial 22 with value: 0.7488688925401696.


Best trial:
FrozenTrial(number=22, state=1, values=[0.7488688925401696], datetime_start=datetime.datetime(2025, 5, 8, 15, 44, 1, 640186), datetime_complete=datetime.datetime(2025, 5, 8, 15, 47, 47, 784565), params={'lr': 0.00046239536595593786, 'batch_size': 16}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.001, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(16, 32, 64))}, trial_id=22, value=None)


['resnet_baseline_optuna_SGD.pkl']

In [5]:


print(study.best_trial)

FrozenTrial(number=22, state=1, values=[0.7488688925401696], datetime_start=datetime.datetime(2025, 5, 8, 15, 44, 1, 640186), datetime_complete=datetime.datetime(2025, 5, 8, 15, 47, 47, 784565), params={'lr': 0.00046239536595593786, 'batch_size': 16}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.001, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(16, 32, 64))}, trial_id=22, value=None)


In [4]:
# import torch
# import optuna
# import numpy as np
# from tsai.models.ResNet import ResNet
# from fastai.learner import Learner
# from fastai.data.core import DataLoaders
# from fastai.metrics import accuracy
# from sklearn.model_selection import KFold
# from torch.utils.data import TensorDataset, Subset
# from fastai.callback.wandb import WandbCallback
# from fastai.data.core import DataLoaders
# from fastai.learner import Learner
# from fastai.metrics import accuracy
# from fastai.losses import CrossEntropyLossFlat
# from sklearn.model_selection import KFold
# from torch.utils.data import Subset
# from tsai.models.ResNet import ResNet

# # Create TensorDataset from your data
# dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

# def objective(trial):
#     print('Trial: ', trial)
#     learning_rate = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
#     batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     accs = []

#     for train_idx, val_idx in kf.split(np.arange(len(dataset))):
#         train_dataset = Subset(dataset, train_idx)
#         val_dataset = Subset(dataset, val_idx)

#         dls = DataLoaders.from_dsets(
#             train_dataset,
#             val_dataset,
#             bs=batch_size,
#             shuffle=True,
#             num_workers=0  # for compatibility and reproducibility
#         )
#         model = ResNet(c_in=1, c_out=2)
#         learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

#         learn.fit_one_cycle(5, lr_max=learning_rate)
#         acc = learn.validate()[1].item()
#         accs.append(acc)

#     return np.mean(accs)

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=25)

# print("Best trial:")
# print(study.best_trial)

# # Save the study if you want
# import joblib
# joblib.dump(study, "resnet_optuna_study.pkl")
